# NIPALS: Non-linear Iterative PArtial Least Squares

In [1]:
import numpy as np
import random

In [2]:
x = np.array([[4,9,6,7,7,8,3,2],
              [6,15,10,15,17,22,9,4],
              [8,21,14,23,27,36,15,6],
              [10,21,14,13,11,10,3,4],
              [12,27,18,21,21,24,9,6],
              [14,33,22,29,31,38,15,8],
              [16,33,22,19,15,12,3,6],
              [18,39,26,27,25,26,9,8],
              [20,45,30,35,35,40,15,10]])
y = np.array([[1, 1],[3, 1],[5, 1],[1, 3],[3, 3],[5, 3],[1 ,5],[3, 5],[5 ,5]])
#X=x[:-1,:]
#Y=y[:-1,:]

In [4]:
X=x
Y=y
print(x.shape,X.shape)
print(y.shape,Y.shape)

(9, 8) (9, 8)
(9, 2) (9, 2)


<img src='./images/nipals.png'>

$$\mathbf{X}_{a}=normalize(\mathbf{X})$$
$$\mathbf{Y}_{a}=normalize(\mathbf{Y})$$

* Arrow 1: Perform 𝐾 regressions, regressing each column from $\mathbf{X}_{a}$ onto the vector $\mathbf{u}_{a}$. The slope coefficients from the regressions are stored as the entries in $\mathbf{w}_{a}$. Columns in $\mathbf{X}_{a}$ which are strongly correlated with $\mathbf{u}_{a}$ will have large weights in $\mathbf{w}_{a}$, while unrelated columns will have small, close to zero, weights. We can perform these regression in one go:
$$\mathbf{w}_{a}=(\mathbf{u}_{a}^{'}\mathbf{u}_{a})^{-1}\mathbf{X}_{a}^{'}\mathbf{u}_{a}$$
* Normalize the weight vector to unit length:
$$\mathbf{w}_{a}=\frac{\mathbf{w}_{a}}{\sqrt{\mathbf{w}_{a}^{'}\mathbf{w}_{a}}}$$
* Regress every row in $\mathbf{x}_{a}$ onto the weight vector. The slope coefficients are stored as entries in $\mathbf{t}_{a}$. This means that rows in $\mathbf{x}_{a}$ that have a similar pattern to that described by the weight vector will have large values in $\mathbf{t}_{a}$. Observations that are totally different to $\mathbf{w}_{a}$ will have near-zero score values. These $N$ regressions can be performed in one go:
$$\mathbf{t}_{a}=(\mathbf{w}_{a}^{'}\mathbf{w}_{a})^{-1}\mathbf{X}_{a}^{'}\mathbf{w}_{a}$$
* Arrow 4 Next, regress every column in $\mathbf{y}_{a}$ onto this score vector, $\mathbf{t}_{a}$. The slope coefficients are stored in $\mathbf{q}_{a}$. We can calculate all $M$ slope coefficients:
$$\mathbf{q}_{a}=(\mathbf{t}_{a}^{'}\mathbf{t}_{a})^{-1}\mathbf{Y}_{a}^{'}\mathbf{t}_{a}$$
Arrow 5 Finally, regress each of the $N$ rows in $\mathbf{y}_{a}$ onto this weight vector, $\mathbf{q}_{a}$. Observations in $\mathbf{y}_{a}$ that are strongly related to $\mathbf{q}_{a}$ will have large positive or negative slope coefficients in vector $\mathbf{u}_{a}$:
$$\mathbf{u}_{a}=(\mathbf{q}_{a}^{'}\mathbf{q}_{a})^{-1}\mathbf{Y}_{a}\mathbf{q}_{a}$$
This is one round of the NIPALS algorithm. We iterate through these 4 arrow steps until the $\mathbf{u}_{a}$ vector does not change much. On convergence, we store these 4 vectors: $\mathbf{w}_{a}$, $\mathbf{t}_{a}$, $\mathbf{q}_{a}$ and $\mathbf{u}_{a}$ which jointly define the $a^{th}$ component.
Then we deflate. Deflation removes variability already explained from $\mathbf{X}_{a}$ and $\mathbf{Y}_{a}$. Deflation proceeds as follows:

Step 1: Calculate a loadings vector for the $X$ space, $\mathbf{p}_{a}$ called using the X-space scores: 
$$\mathbf{p}_{a}=(\mathbf{t}_{a}^{'}\mathbf{t}_{a})^{-1}\mathbf{X}_{a}^{'}\mathbf{t}_{a}$$
Step 2: Remove the predicted variability from $X$ and $Y$ Using the loadings, $\mathbf{p}_{a}$ just calculated
above, we remove from $\mathbf{X}_{a}$ the best prediction of $\mathbf{p}_{a}$, in other words, remove everything we can explain about it. We also remove any variance explained from $\mathbf{Y}_{a}$:
$$\mathbf{\hat{X}}_{a}=\mathbf{t}_{a}\mathbf{p}_{a}^{'}$$
$$\mathbf{\hat{Y}}_{a}=\mathbf{u}_{a}\mathbf{q}_{a}^{'}$$
$$\mathbf{X}_{a}=\mathbf{X}_{a}-\mathbf{\hat{X}}_{a}$$
$$\mathbf{Y}_{a}=\mathbf{Y}_{a}-\mathbf{\hat{Y}}_{a}$$

In [5]:
def pls(X, Y, no_components):
    MAXITER=10000
    CC=0.000000001
    Xc = (X-X.mean(axis=0))/X.std(axis=0,ddof=1)
    Yc = (Y-Y.mean(axis=0))/Y.std(axis=0,ddof=1)
    data_samples,X_variables = X.shape
    data_samples,Y_variables = Y.shape
    
    W = np.empty((X_variables, no_components))
    T = np.empty((data_samples, no_components))
    Q = np.empty((Y_variables, no_components))
    U = np.empty((data_samples, no_components))
    P = np.empty((X_variables, no_components))
    c = np.empty((no_components,))
    components = 0
    Xa = Xc
    Ya = Yc

    for j in range(0, no_components):
        ua = Ya[:,0]
        ITER = 0
        delU = CC * 10.0

        while ITER < MAXITER and delU > CC:                      
            wa = np.dot(Xa.T,ua)/np.dot(ua.T,ua)
            wa = wa/np.linalg.norm(wa, 2)
            ta = np.dot(Xa,wa)/np.dot(wa.T,wa)
            qa = np.dot(Ya.T,ta)/np.dot(ta.T,ta)
            qa = qa/np.linalg.norm(qa, 2)
            old_ua = ua
            ua = np.dot(Ya,qa)/np.dot(qa.T,qa)
            delU = np.linalg.norm(ua - old_ua)
            ITER += 1

        if ITER >= MAXITER:
            if ignore_failures:
                break
            else:
                raise ConvergenceError('PLS failed to converge for ' 'component: ' '{}'.format(components+1))
        W[:, j] = wa
        T[:, j] = ta
        Q[:, j] = qa
        U[:, j] = ua

        P[:, j] = np.dot(Xa.T,ta) / np.dot(ta.T,ta)
        Xa = Xa - np.outer(ta, P[:, j].T)
        Ya = Ya -  np.outer(ta, qa.T)
    return T,W,U,Q,P

In [6]:
[T,W,U,Q,P] = pls(X,Y,2)

In [7]:
Q

array([[ 0.76964905,  0.63846718],
       [ 0.63846718, -0.76964905]])

In [8]:
U

array([[-1.62595257,  0.15147577],
       [-0.7372384 ,  0.88871417],
       [ 0.15147577,  1.62595257],
       [-0.88871417, -0.7372384 ],
       [ 0.        ,  0.        ],
       [ 0.88871417,  0.7372384 ],
       [-0.15147577, -1.62595257],
       [ 0.7372384 , -0.88871417],
       [ 1.62595257, -0.15147577]])

In [9]:
T

array([[-4.18022816,  0.17860415],
       [-1.89539644,  1.0478774 ],
       [ 0.38943527,  1.91715065],
       [-2.28483172, -0.86927325],
       [ 0.        ,  0.        ],
       [ 2.28483172,  0.86927325],
       [-0.38943527, -1.91715065],
       [ 1.89539644, -1.0478774 ],
       [ 4.18022816, -0.17860415]])

In [10]:
W

array([[ 0.33026333, -0.44801536],
       [ 0.35600191, -0.34167241],
       [ 0.35600191, -0.34167241],
       [ 0.38849571,  0.04154404],
       [ 0.37018428,  0.26033672],
       [ 0.33147652,  0.44373521],
       [ 0.29936472,  0.54148971],
       [ 0.38728564, -0.07867023]])

In [11]:
U

array([[-1.62595257,  0.15147577],
       [-0.7372384 ,  0.88871417],
       [ 0.15147577,  1.62595257],
       [-0.88871417, -0.7372384 ],
       [ 0.        ,  0.        ],
       [ 0.88871417,  0.7372384 ],
       [-0.15147577, -1.62595257],
       [ 0.7372384 , -0.88871417],
       [ 1.62595257, -0.15147577]])